In [1]:
import pandas as pd
from pandas import json_normalize

# Load the Excel file
file_path = 'E:\GUVI\PROJECT 3 CARDHEKO\DATASET\cars_data_combined.xlsx'
df = pd.read_excel(file_path, sheet_name=None)  # Load all sheets

# Assume the relevant sheet is named 'Sheet1'
df = df['Sheet1']

# Define the columns containing key-value pairs
columns_with_pairs = ['new_car_detail']

# Function to process each column
def process_column(column):
    # Convert the column to a DataFrame with flattened dictionaries
    flattened_dfs = df[column].apply(lambda x: pd.json_normalize(eval(x)) if pd.notnull(x) else pd.DataFrame())
    combined_df = pd.concat(flattened_dfs.tolist(), ignore_index=True)
    return combined_df

# Process each column and combine the results
result_dfs = [process_column(col) for col in columns_with_pairs]
# Remove duplicates if any
#expanded_df = expanded_df.drop_duplicates()
# Combine all the processed DataFrames
expanded_df = pd.concat(result_dfs, axis=1)

# Include the City column
df_final_details = pd.concat([df[['City']], expanded_df], axis=1)

# Display the final DataFrame
print(df_final_details)

           City  it      ft         bt        km transmission  ownerNo  \
0     Bangalore   0  Petrol  Hatchback  1,20,000       Manual        3   
1     Bangalore   0  Petrol        SUV    32,706       Manual        2   
2     Bangalore   0  Petrol  Hatchback    11,949       Manual        1   
3     Bangalore   0  Petrol      Sedan    17,794       Manual        1   
4     Bangalore   0  Diesel        SUV    60,000       Manual        1   
...         ...  ..     ...        ...       ...          ...      ...   
6879    Kolkata   0  Petrol  Hatchback    10,000       Manual        1   
6880    Kolkata   0  Petrol  Hatchback  1,20,000       Manual        1   
6881    Kolkata   0  Petrol      Sedan    50,000    Automatic        3   
6882    Kolkata   0  Petrol  Hatchback    40,000       Manual        1   
6883    Kolkata   0  Diesel        SUV  1,20,000       Manual        2   

          owner            oem                  model  modelYear  \
0     3rd Owner         Maruti         Maru

In [3]:
df_final_details.shape

(6884, 20)

In [5]:
import pandas as pd
import ast
from pandas import json_normalize

# Load the Excel file and read only the necessary column
file_path = 'E:\GUVI\PROJECT 3 CARDHEKO\DATASET\cars_data_combined.xlsx'
df = pd.read_excel(file_path, usecols=['new_car_overview'])

def extract_data_from_row(row):
    cell_data = row['new_car_overview']
    if pd.notnull(cell_data):
        try:
            # Convert the string to a dictionary
            data_dict = ast.literal_eval(cell_data)
            
            # Extract 'top' section
            top_data = pd.DataFrame()
            if 'top' in data_dict:
                top_data = json_normalize(data_dict['top'])
            
            # Return the extracted data
            return top_data
        except Exception as e:
            print(f"Error processing row: {row}")
            print(f"Exception: {e}")
            return pd.DataFrame()
    return pd.DataFrame()

# Process data in batches to manage memory and CPU usage
chunk_size = 1000  # Adjust chunk size based on your system's memory
all_data = []

for start in range(0, len(df), chunk_size):
    end = min(start + chunk_size, len(df))
    chunk_df = df.iloc[start:end]
    
    # Extract and normalize data for each chunk
    chunk_data = [extract_data_from_row(row) for _, row in chunk_df.iterrows()]
    chunk_combined = pd.concat(chunk_data, ignore_index=True) if chunk_data else pd.DataFrame()
    
    if not chunk_combined.empty:
        all_data.append(chunk_combined)

# Combine all chunks
df_combined = pd.concat(all_data, ignore_index=True) if all_data else pd.DataFrame()

# Debugging: Print the shape and sample data
print("Shape of df_combined:", df_combined.shape)
print(df_combined.head())

# Add a unique identifier column for each row
df_combined['id'] = df_combined.groupby('key').cumcount()

# Pivot the DataFrame
df_final_overview = df_combined.pivot_table(index='id', columns='key', values='value', aggfunc='first')

# Reset the index to get a clean DataFrame
df_final_overview = df_final_overview.reset_index(drop=True)

# Drop the 'id' column if needed
df_final_overview = df_final_overview.drop(columns=['id'], errors='ignore')

# Debugging: Print the shape and sample data of the final DataFrame
print("Shape of df_final_overview:", df_final_overview.shape)
print(df_final_overview.head())


Shape of df_combined: (67988, 3)
                  key                  value  \
0   Registration Year                   2015   
1  Insurance Validity  Third Party insurance   
2           Fuel Type                 Petrol   
3               Seats                5 Seats   
4          Kms Driven           1,20,000 Kms   

                                                icon  
0  https://images10.gaadi.com/listing/vdp/co/v1/r...  
1  https://images10.gaadi.com/listing/vdp/co/v1/i...  
2  https://images10.gaadi.com/listing/vdp/co/v1/f...  
3  https://images10.gaadi.com/listing/vdp/co/v1/s...  
4  https://images10.gaadi.com/listing/vdp/co/v1/k...  
Shape of df_final_overview: (6884, 10)
key Engine Displacement Fuel Type     Insurance Validity    Kms Driven  \
0                998 cc    Petrol  Third Party insurance  1,20,000 Kms   
1               1497 cc    Petrol          Comprehensive    32,706 Kms   
2               1199 cc    Petrol          Comprehensive    11,949 Kms   
3            

In [7]:
import pandas as pd
import ast
from pandas import json_normalize

# Load the Excel file and read only the necessary column
file_path = 'E:\\GUVI\\PROJECT 3 CARDHEKO\\DATASET\\cars_data_combined.xlsx'
df = pd.read_excel(file_path, usecols=['new_car_specs'])

# Add a unique identifier for each row (car) to maintain consistency
df['car_id'] = df.index

def extract_data_from_row(row):
    cell_data = row['new_car_specs']
    if pd.notnull(cell_data):
        try:
            # Convert the string to a dictionary
            data_dict = ast.literal_eval(cell_data)
            
            # Extract 'top' section
            top_data = pd.DataFrame()
            if 'top' in data_dict:
                top_data = json_normalize(data_dict['top'])
            
            # Extract 'list' section from nested dictionaries in 'data'
            list_data = pd.DataFrame()
            if 'data' in data_dict:
                for data_entry in data_dict['data']:
                    if 'list' in data_entry:
                        list_data = pd.concat([list_data, json_normalize(data_entry['list'])], ignore_index=True)
            
            # Combine 'top' and 'list' data
            combined_data = pd.concat([top_data, list_data], ignore_index=True) if not top_data.empty or not list_data.empty else pd.DataFrame()
            
            # Add car_id to combined data to associate with the correct car
            combined_data['car_id'] = row['car_id']
            
            # Return the combined data
            return combined_data
        except Exception as e:
            print(f"Error processing row: {row}")
            print(f"Exception: {e}")
            return pd.DataFrame()
    return pd.DataFrame()

# Process data in batches to manage memory and CPU usage
chunk_size = 1000  # Adjust chunk size based on your system's memory
all_data = []

for start in range(0, len(df), chunk_size):
    end = min(start + chunk_size, len(df))
    chunk_df = df.iloc[start:end]
    
    # Extract and normalize data for each chunk
    chunk_data = [extract_data_from_row(row) for _, row in chunk_df.iterrows()]
    chunk_combined = pd.concat(chunk_data, ignore_index=True) if chunk_data else pd.DataFrame()
    
    if not chunk_combined.empty:
        all_data.append(chunk_combined)

# Combine all chunks
df_combined = pd.concat(all_data, ignore_index=True) if all_data else pd.DataFrame()

# Debugging: Print the shape and sample data
print("Shape of df_combined:", df_combined.shape)
print(df_combined.head())

# Pivot the DataFrame using 'car_id' as index to avoid duplicate rows
df_final_specs = df_combined.pivot_table(index='car_id', columns='key', values='value', aggfunc='first')

# Reset the index to get a clean DataFrame
df_final_specs = df_final_specs.reset_index(drop=True)

# Debugging: Print the shape and sample data of the final DataFrame
print("Shape of df_final_specs:", df_final_specs.shape)
print(df_final_specs.head())


Shape of df_combined: (233769, 3)
         key      value  car_id
0    Mileage  23.1 kmpl       0
1     Engine     998 CC       0
2  Max Power   67.04bhp       0
3     Torque       90Nm       0
4      Seats          5       0
Shape of df_final_specs: (6884, 40)
key   Acceleration Alloy Wheel Size  BoreX Stroke Cargo Volumn   Color  \
0    15.05 Seconds              NaN    73 X 82 mm   235-litres   White   
1              NaN               16  79 X 76.5 mm   352-litres   White   
2     14.3 Seconds               14  77 X 85.8 mm   242-litres     Red   
3     14.2 Seconds               14           NaN   407-litres  Others   
4       12 Seconds               16  69.6 x 82 mm   353-litres    Gray   

key Compression Ratio Displacement Drive Type   Engine  \
0              11.0:1          998        FWD   998 CC   
1              11.0:1         1497        FWD  1497 CC   
2              10.8:1         1199        FWD  1199 CC   
3                 NaN         1197       FWD   1197 CC   
4  

In [9]:
df_cars_combined=pd.concat([df_final_details,df_final_overview,df_final_specs], axis=1)
print(df_cars_combined.shape)

(6884, 70)


In [11]:
df_cars_combined.tail(10)

,City,it,ft,bt,km,transmission,ownerNo,owner,oem,model,...,Top Speed,Torque,Turbo Charger,Turning Radius,Tyre Type,Value Configuration,Values per Cylinder,Wheel Base,Wheel Size,Width
6874,Kolkata,0,Petrol,Sedan,"60,000",Manual,1,1st Owner,Maruti,Maruti Swift Dzire,...,NaN,114Nm,No,4.7m,"Tubeless,Radial",SOHC,4,2390mm,14,1690mm
6875,Kolkata,0,Petrol,Hatchback,"30,000",Manual,3,3rd Owner,Maruti,Maruti 800,...,144km/hr,59Nm,No,4.4metres,Radial,SOHC,2,2175mm,12,1440mm
6876,Kolkata,0,Petrol,Sedan,"60,000",Manual,1,1st Owner,Honda,Honda City,...,NaN,146Nm,No,5.3metres,"Tubeless,Radial",SOHC,4,2550mm,15,1695mm
6877,Kolkata,0,Petrol,Hatchback,"70,000",Manual,1,1st Owner,Volkswagen,Volkswagen Polo,...,165 kmph,110Nm,No,4.97 metres,"Tubeless,Radial",DOHC,4,2469mm,NaN,1682mm
6878,Kolkata,0,Petrol,Hatchback,"30,000",Manual,1,1st Owner,Hyundai,Hyundai Santro Xing,...,141km/hr,96.1Nm,No,4.4m,"Tubeless,Radial",SOHC,3,2380mm,13,1525mm
6879,Kolkata,0,Petrol,Hatchback,"10,000",Manual,1,1st Owner,Maruti,Maruti Celerio,...,NaN,89Nm,NaN,NaN,"Tubeless, Radial",NaN,4,2435,NaN,1655
6880,Kolkata,0,Petrol,Hatchback,"1,20,000",Manual,1,1st Owner,Maruti,Maruti Alto 800,...,140 kmph,69Nm,No,4.6 metres,Tubeless Tyres,DOHC,4,2360mm,NaN,1490mm
6881,Kolkata,0,Petrol,Sedan,"50,000",Automatic,3,3rd Owner,Mercedes-Benz,Mercedes-Benz C-Class,...,230km/hr,285Nm,No,5.42 metres,"Tubeless,Radial",NaN,4,2760mm,17,1770mm
6882,Kolkata,0,Petrol,Hatchback,"40,000",Manual,1,1st Owner,Maruti,Maruti Ritz,...,156 Kmph,114Nm,No,4.7 metres,"Tubeless,Radial",DOHC,4,2360mm,14,1680mm
6883,Kolkata,0,Diesel,SUV,"1,20,000",Manual,2,2nd Owner,Renault,Renault Duster,...,168 Kmph,245Nm,Yes,5.2 metres,"Tubeless,Radial",DOHC,4,2673mm,16,1822mm


In [65]:
df_cars_combined.columns

Index(['City', 'it', 'ft', 'bt', 'km', 'transmission', 'ownerNo', 'owner',
       'oem', 'model', 'modelYear', 'centralVariantId', 'variantName', 'price',
       'priceActual', 'priceSaving', 'priceFixedText', 'trendingText.imgUrl',
       'trendingText.heading', 'trendingText.desc', 'Engine Displacement',
       'Fuel Type', 'Insurance Validity', 'Kms Driven', 'Ownership', 'RTO',
       'Registration Year', 'Seats', 'Transmission', 'Year of Manufacture',
       'Acceleration', 'Alloy Wheel Size', 'BoreX Stroke', 'Cargo Volumn',
       'Color', 'Compression Ratio', 'Displacement', 'Drive Type', 'Engine',
       'Engine Type', 'Front Brake Type', 'Front Tread', 'Fuel Suppy System',
       'Gear Box', 'Gross Weight', 'Ground Clearance Unladen', 'Height',
       'Kerb Weight', 'Length', 'Max Power', 'Max Torque', 'Mileage',
       'No Door Numbers', 'No of Cylinder', 'Rear Brake Type', 'Rear Tread',
       'Seating Capacity', 'Seats', 'Steering Type', 'Super Charger',
       'Top Speed', 

In [78]:

#df_cars_combined.to_csv('E:\GUVI\PROJECT 3 CARDHEKO\DATASET\cars_combined.csv', index=False)

In [13]:
df_cars_combined.to_pickle('E:\\GUVI\\PROJECT 3 CARDHEKO\\DATASET\\cars_combined.pkl')
